In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

from langchain.document_loaders import TextLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load Sentence Transformers Embedding model
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
from langchain.vectorstores import Chroma

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model_name = "microsoft/phi-3"  # Updated to "phi-3" as it is now available

C:\Users\anant\AppData\Local\Temp\ipykernel_2640\282517333.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
c:\Users\anant\Documents\Anaconda-Installation\envs\genai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

all_documents = []
data_folder = "Documents/"

for file_name in os.listdir(data_folder):
    file_path = os.path.join(data_folder, file_name)
    
    if file_name.endswith(".txt"):
        loader = TextLoader(file_path)
    elif file_name.endswith(".pdf"):
        loader = PyMuPDFLoader(file_path)
    else:
        continue  # skip unsupported file types

    docs = loader.load()
    all_documents.extend(docs)

print(f"Total Documents Loaded: {len(all_documents)}")


Total Documents Loaded: 113


In [3]:
# Initialize splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # size of each chunk
    chunk_overlap=50     # overlapping to preserve context
)

# Split all loaded documents
chunks = text_splitter.split_documents(all_documents)

print(f"Total Chunks Created: {len(chunks)}")
print("Sample Chunk Preview:")
print(chunks[0].page_content[:500])  # Show first chunk's first 500 characters

Total Chunks Created: 452
Sample Chunk Preview:
AI For Managers


In [4]:
from langchain.vectorstores import Chroma

# Create a new ChromaDB database
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"  # Directory where vector DB will be saved
)

# Persist the database to disk
vectorstore.persist()

print("✅ Vectorstore created and saved successfully.")


✅ Vectorstore created and saved successfully.


C:\Users\anant\AppData\Local\Temp\ipykernel_2640\909237655.py:11: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
# Reload the persisted ChromaDB
vectorstore = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embeddings  # reuse the same embeddings
)

# Create a retriever
retriever = vectorstore.as_retriever()


In [5]:
# Use Qwen 0.5B model
model_name = "Qwen/Qwen2-0.5B"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto",
    use_auth_token=hf_token,
    trust_remote_code=True
)

# Build the text generation pipeline
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.3,
    do_sample=True
)

# Wrap into LangChain LLM
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=llm_pipeline)

c:\Users\anant\Documents\Anaconda-Installation\envs\genai\Lib\site-packages\transformers\models\auto\tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
c:\Users\anant\Documents\Anaconda-Installation\envs\genai\Lib\site-packages\transformers\models\auto\auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Device set to use cuda:0
C:\Users\anant\AppData\Local\Temp\ipykernel_2640\1709709670.py:26: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip in

In [ ]:
# Build the RetrievalQA Chain
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"  # simplest chain type
)


In [6]:
def run_cli_qa():
    print("\n🧠 GenAI Q&A Engine Started! (Type 'exit' to quit)\n")
    
    while True:
        query = input("Ask your question: ")
        
        if query.lower() in ['exit', 'quit']:
            print("\n👋 Exiting the Q&A Engine. Goodbye!")
            break
        
        if query.strip() == "":
            print("⚠️ Please enter a valid question.\n")
            continue
        
        try:
            answer = qa_chain.run(query)
            print(f"\n📝 Answer: {answer}\n")
        
        except Exception as e:
            print(f"❌ Error: {str(e)}\n")




In [ ]:
run_cli_qa("What is AI for Managers all about?", qa_chain=qa_chain)

TypeError: run_cli_qa() takes 0 positional arguments but 1 was given